In [3]:
from traffic.core import Traffic, Flight

from traffic.data.samples import quickstart
from traffic.data.datasets import squawk7700
from traffic.data import airports

import pandas as pd
import time

Output()

C:\Users\tobso\Desktop\programering\masters-thesis\venv\Lib\site-packages\traffic\data\datasets\zenodo.py:82: TqdmExperimentalWarning: rich is experimental/alpha
  with tqdm(


Output()

In [4]:
def load_flights_from_csv(csv_file):
    # Load the flight data from the CSV file into a Traffic object
    traffic = Traffic.from_file(csv_file)
    
    if traffic is None:
        print("Failed to load flight data.")
        return
    
    traffic.data["timestamp"] = pd.to_datetime(traffic.data['timestamp'], errors='coerce', utc=True, unit='s')
    traffic.data["lastcontact"] = pd.to_datetime(traffic.data['lastcontact'], errors='coerce', utc=True, unit='s')
    traffic.data["lastposupdate"] = pd.to_datetime(traffic.data['lastposupdate'], errors='coerce', utc=True, unit='s')
    traffic.data.dropna(subset=['longitude', 'latitude'], inplace=True)
    
    return traffic

In [5]:
flights = load_flights_from_csv(csv_file="data/states_2022-02-21-16.csv")

In [6]:
flights[1].emergency

<bound method NavigationFeatures.emergency of Flight(icao24='008080', callsign='LNK909  ')>

In [7]:
quickstart.airborne()

,,count
icao24,callsign,
39b002,FHMAC,3283
3aabfc,FMY8055,2133
39c82b,PEA501,1891
4bc844,PGT90Y,1599
4d02be,JFA12P,1591
3d7009,DFORH,1559
3985a4,AFR19BH,1547
3944f1,AFR15AH,1546
3999e4,PEA302,1541


In [8]:
quickstart.query('callsign.str.startswith("AFR")').first("10 min").eval()

,,count
icao24,callsign,
394c13,AFR1753,600
3951c1,AFR6724,600
398567,AFR15XV,600
398564,AFR9455,600
3965af,AFR276,600
3965a5,AFR4145,600
3965a3,AFR072,600
39644d,AFR914,600
396441,AFR104,600


In [9]:
def select_landing(airport: "Airport"):
    for flight in quickstart:
        if low_alt := flight.query("altitude < 3000"):         # Flight -> None or Flight
            if low_alt.last("2 min").vertical_rate_mean < -500:# Flight -> bool
                if low_alt.first("2 min").vertical_rate_mean < 500:# Flight -> bool
                    if low_alt.intersects(airport):                # Flight -> bool
                        if low_alt.aligned_on_ils(airport).has():  # Flight -> bool
                            yield low_alt          # Flight -> None or Flight

In [10]:
Traffic.from_flights(select_landing(airports["LFPO"]))[3].map_leaflet()

airports @ourairports.com:   0%|          | 0/3546 [00:00<?, ?it/s]

runways @ourairports.com:   0%|          | 0/1260 [00:00<?, ?it/s]

Map(center=[48.69000510556506, 2.232647785496943], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [11]:
quickstart["3964f4"].map_leaflet()

Several callsigns for one flight, consider splitting
Several callsigns for one flight, consider splitting


Map(center=[48.537532136850984, 2.335286307940885], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [12]:
ids = squawk7700.query("diverted == diverted").flight_ids

In [20]:
from ipywidgets import Layout
squawk7700[ids[7]].map_leaflet(
    highlight=dict(red=lambda f: f.emergency()),
    layout=Layout(height="500px", max_width="800px"),
)

C:\Users\tobso\Desktop\programering\masters-thesis\venv\Lib\site-packages\traffic\core\flight.py:1411: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return Position(self.data.ffill().iloc[-1])


Map(center=[47.360406121636586, 7.428182574806808], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [23]:
squawk7700[1].data

,timestamp,altitude,callsign,flight_id,groundspeed,icao24,latitude,longitude,squawk,track,...,landing,destination,diverted,tweet_problem,tweet_result,tweet_fueldump,avh_id,avh_problem,avh_result,avh_fueldump
2511536,2019-05-20 15:23:04+00:00,34000.0,AAL1188,AAL1188_20190520,451.246771,ad4747,29.191372,-94.628152,4752,337.750976,...,KDFW,KDFW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2511537,2019-05-20 15:23:05+00:00,34000.0,AAL1188,AAL1188_20190520,451.246771,ad4747,29.193514,-94.629122,4752,337.750976,...,KDFW,KDFW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2511538,2019-05-20 15:23:06+00:00,34000.0,AAL1188,AAL1188_20190520,451.246771,ad4747,29.195469,-94.630037,4752,337.750976,...,KDFW,KDFW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2511539,2019-05-20 15:23:07+00:00,34000.0,AAL1188,AAL1188_20190520,451.246771,ad4747,29.197377,-94.630953,4752,337.750976,...,KDFW,KDFW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2511540,2019-05-20 15:23:08+00:00,34000.0,AAL1188,AAL1188_20190520,451.624454,ad4747,29.199326,-94.631847,4752,337.750976,...,KDFW,KDFW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2514888,2019-05-20 16:19:11+00:00,945.0,AAL1188,AAL1188_20190520,134.033462,ad4747,32.927189,-97.054579,4752,180.848764,...,KDFW,KDFW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2514889,2019-05-20 16:19:12+00:00,935.0,AAL1188,AAL1188_20190520,134.033462,ad4747,32.926491,-97.054602,4752,180.848764,...,KDFW,KDFW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2514890,2019-05-20 16:19:13+00:00,925.0,AAL1188,AAL1188_20190520,134.033462,ad4747,32.925793,-97.054624,4752,180.848764,...,KDFW,KDFW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2514891,2019-05-20 16:19:14+00:00,912.5,AAL1188,AAL1188_20190520,134.524080,ad4747,32.925350,-97.054624,4752,180.848764,...,KDFW,KDFW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
flights[2].map_leaflet(
    highlight=dict(red=lambda f: f.emergency()),
    layout=Layout(height="500px", max_width="800px"),
)

Map(center=[-25.620562845126965, 27.780781399647957], controls=(ZoomControl(options=['position', 'zoom_in_text…

Note: you may need to restart the kernel to use updated packages.
